In [1]:
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
%matplotlib inline


class JacksCarRentalEnvironment:
    
    TRANSFER_COST = 2
    RENTAL_INCOME = 10
    MAX_CAPACITY = 20
    REQUEST_RATE = (3,4)
    RETURN_RATE = (3,2)
    
    def __init__(self):
        self._carparkA = CarPark(3,3,self.__class__.MAX_CAPACITY)
        self._carparkB = CarPark(4,2,self.__class__.MAX_CAPACITY)
        print([self._carparkA.getObservation(),self._carparkB.getObservation()])
        
    def reset(self,val=None):
        self._carparkA = CarPark(3,3,self.__class__.MAX_CAPACITY,val)
        self._carparkB = CarPark(4,2,self.__class__.MAX_CAPACITY,val)
        print([self._carparkA.getObservation(),self._carparkB.getObservation()])
        
        
    def step(self,val):
        # val > 0 Autos wandern von A nach B
        # val < 0 Autos wandern von B nach A
        if val < 0:
            a = self._carparkB
            b = self._carparkA
            val = -1*val
        else:
            a = self._carparkA
            b = self._carparkB
        # Zu aller erst werden die autos von jack bewegt    
        # An einem Tag kommen zuerst Kunden und leihen Autos
        # Dann werden Autos zurückgegeben
        # Dann kann jack entscheiden welche autos bewegt werden
        # Einfachster Fall val == 0
        carsRented = 0
        reward = 0.0
        done = False
        if val == 0:
            carsRented += a.carsRenting()
            carsRented += b.carsRenting()
            reward = 10.0*carsRented
            # Bevor Autos kommen auf available pruefen oder spaeter
            if 0 in [self._carparkA.getObservation(),self._carparkB.getObservation()]:
                done = True
            a.carsArriving()
            b.carsArriving()
        else:
            availableCars = a.removeCars(val)
            reward = -2.0*val
            b.addCars(availableCars)
            carsRented += a.carsRenting()
            carsRented += b.carsRenting()
            reward = reward + 10.0*carsRented
            if 0 in [self._carparkA.getObservation(),self._carparkB.getObservation()]:
                done = True
            a.carsArriving()
            b.carsArriving()
        observation = [self._carparkA.getObservation(),self._carparkB.getObservation()]
        return (observation,reward,done,'')
    
    
    
class CarPark:
    
    def __init__(self, lamRequest, lamReturn, maxSize,val=None):
        self._lamRequest = lamRequest
        self._lamReturn = lamReturn
        self._maxSize = maxSize
        self._maxMove = 5
        if val == None:
            self._carsAvailable = np.random.randint(0,21,1)[0]
        else:
            self._carsAvailable = val
        
        
        
    def getObservation(self):
        return self._carsAvailable
    
    def carsArriving(self):
        r = np.random.poisson(self._lamReturn)
        if self._carsAvailable + r > self._maxSize:
            self._carsAvailable = self._maxSize
        else:
            self._carsAvailable += r
    
    def carsRenting(self):
        carsRented = np.random.poisson(self._lamRequest)
        if self._carsAvailable - carsRented < 0:
            carsRented = self._carsAvailable
            self._carsAvailable = 0
        else:
            self._carsAvailable -= carsRented
        return carsRented
           
    def addCars(self,amount):
        if amount > self._maxMove:
            raise Exception('Zu viele Autos')
        elif amount < 0:
            raise Exception('Du wolltest autos hinzufuegen')
        self._carsAvailable += amount
        if self._carsAvailable > self._maxSize:
            self._carsAvailable = self._maxSize
            
    def removeCars(self,amount):
        if amount > self._maxMove:
            raise Exception('Zu viele Autos')
        elif amount < 0:
            raise Exception('Du')
        if self._carsAvailable - amount < 0:
            amount = self._carsAvailable
            self._carsAvailable = 0
        else:
            self._carsAvailable -= amount
        return amount

In [2]:
class JacksCarRentalEnvironmentModel(JacksCarRentalEnvironment):
    
    def get_transition_probabilities_and_expected_reward(self, state, action):
        """
            Compute the $p(s', r\mid s,a)$
            Parameters
            ----------
            old_state: tuple of two ints
                the state (cars_at_A, cars_at_B)
            action: int
                nigthly movements of the cars as a int between -5 to 5, e.g.: 
                action +3: move three cars from A to B. 
                action -2: move two cars from B to A.
                
            Returns
            -------
            numpy array (2d - float): mapping from (new) states to probabilities
                index first dimension: cars at A
                index second dimension: cars at B
                value: probability  
            float:  expected reward for the state-action pair
        """
        #assert type(action) == int
        assert np.abs(action) <=5
        # first we move the cars in the night
        num_states_for_a_location = self.__class__.MAX_CAPACITY + 1
            
        state = self._nightly_moves(state, action)
    
        expected_reward = - self.__class__.TRANSFER_COST * np.abs(action)
        expected_reward += self._expected_reward_rent(state)
        
        transition_probabilities = self._rent_transition_probabilities(state)
        transition_probabilities = self._returns_transition_probabilities(transition_probabilities)
        return transition_probabilities, expected_reward  
        
        
    def _nightly_moves(self, state, action):
        
        cars_at_A = state[0]
        cars_at_B = state[1]
        if action > 0:
            cars_moved = min(action, cars_at_A)
        else:
            cars_moved = max(action, -cars_at_B)
        
        cars_at_A = min(cars_at_A - cars_moved, self.__class__.MAX_CAPACITY)
        cars_at_B = min(cars_at_B + cars_moved, self.__class__.MAX_CAPACITY)
        return [cars_at_A, cars_at_B]
        
    def _expected_reward_rent(self, state):
        expected_reward_rent = 0.
        m = self.__class__.MAX_CAPACITY+1
        request_mu = self.__class__.REQUEST_RATE
        for i in (0,1):
            cars_at_loc = state[i]
            rv = scipy.stats.poisson(request_mu[i])
            rent_prob = (rv.pmf(range(m)))
            #logger.debug(rent_prob)
            rent_prob[cars_at_loc] = rent_prob[cars_at_loc:].sum()
            rent_prob[cars_at_loc+1:] = 0. 
            #logger.debug(rent_prob)
            expected_reward_rent += np.dot(np.arange(len(rent_prob)), rent_prob) * self.__class__.RENTAL_INCOME
        return expected_reward_rent
        
    def _rent_transition_probabilities(self, state):
        
        num_states_for_a_location = self.__class__.MAX_CAPACITY + 1
        m = 15
        n = num_states_for_a_location + 2*m
        p_  = [np.zeros(n), np.zeros(n)]
        request_mu = self.__class__.REQUEST_RATE
        
        for i in (0,1):
            rv = scipy.stats.poisson(request_mu[i])
            cars_at_loc = state[i]
            x = cars_at_loc + m + 1     
            rent_prob = (rv.pmf(range(x)))
            assert state[i]-x+m+1 == 0
            p_[i][0:cars_at_loc+m+1] = rent_prob[::-1]
            p_[i][m] = p_[i][:m+1].sum()
            p_[i] = p_[i][m:-m]
        return p_
    
    def _returns_transition_probabilities(self, state_probalility):
        
        num_states_for_a_location = self.__class__.MAX_CAPACITY + 1
        m=11
        n = num_states_for_a_location + 2*m
        returns_mu = self.__class__.RETURN_RATE
        p_  = [np.zeros(num_states_for_a_location), np.zeros(num_states_for_a_location)]
        for i in (0,1):
            rv = scipy.stats.poisson(returns_mu[i])
            #logger.debug(len(state_probalility[i]))
            for cars_at_loc in range(len(state_probalility[i])):
                p = np.zeros(n)
                #logger.debug(p.shape)
                x = num_states_for_a_location - cars_at_loc + m - 1     
                return_prob = (rv.pmf(range(x)))
                #logger.debug (p[cars_at_loc+m:-1].shape)
                p[cars_at_loc+m:-1] = return_prob
                #logger.debug(return_prob)
                p[num_states_for_a_location+m-1] = p[num_states_for_a_location+m-1:].sum()
                p = p[m:-m]
                #logger.debug(p)
                #logger.debug(p.sum())
                p_[i] += p * state_probalility[i][cars_at_loc]
        return p_
    

# Iterative Policy Evaluation

d.h. Policy gegeben und gesucht ist v(s) 

v (s)= sum(π(a|s) Ra+γ P(s′))

v(s) = Summe ueber alles moeglichen Aktions
k+1 s ss k a∈A s′∈S
vk+1 = Rπ + γPπvk

In [3]:
jack = JacksCarRentalEnvironmentModel()

[3, 11]


In [4]:
p,r = jack.get_transition_probabilities_and_expected_reward((10,10),4)

In [5]:
sum(p[1])


0.99999999999841072

In [6]:
p

[array([  4.17802832e-03,   1.75535581e-02,   4.22253355e-02,
          7.76405072e-02,   1.18952063e-01,   1.54141420e-01,
          1.67806208e-01,   1.52576096e-01,   1.16179533e-01,
          7.48067546e-02,   4.12597848e-02,   1.97579560e-02,
          8.31968606e-03,   3.11595176e-03,   1.04850691e-03,
          3.19796120e-04,   8.90905330e-05,   2.28223284e-05,
          5.40770353e-06,   1.19135234e-06,   3.02813622e-07]),
 array([  1.03299277e-05,   4.73734715e-05,   1.60906340e-04,
          5.01296730e-04,   1.45331762e-03,   3.88040298e-03,
          9.45596609e-03,   2.08515568e-02,   4.12180332e-02,
          7.22527829e-02,   1.10924985e-01,   1.47051374e-01,
          1.65751023e-01,   1.56401198e-01,   1.21975811e-01,
          7.82005992e-02,   4.14377557e-02,   1.83814218e-02,
          6.93281151e-03,   2.25789937e-03,   8.53155607e-04])]

In [7]:
v_old = np.zeros((21,21)) # axis 0 = State A , axis 1 = State B  --> Alle States keinen Value
policiy = np.zeros((21,21,len(actions)))
policiy.fill(1/len(actions))  # Policy gleichverteilt --> alle Aktions gleich wahrscheinlich

NameError: name 'actions' is not defined

In [ ]:
def updateV(s,v_Old,policiy):
    # s ist (a,b)
    # γ = 1 = y
    # Summe ueber alle Aktions
    y = 1
    temp = 0 # sum
    actions = np.arange(-5,6,1)
    pol_forState = policiy[s[0],s[1]]
    for i in range(len(actions)):
        act = actions[i]
        pol_a_s = pol_forState[i] 
        stateprobs, r_a = jack.get_transition_probabilities_and_expected_reward(s,act)
        # Summe ueber alle S'
        temp_a = 0
        for carA in range(len(stateprobs[0])):
            for carB in range(len(stateprobs[1])):
                prob_of_State_prime = stateprobs[0][carA] * stateprobs[1][carB]
                temp_a += prob_of_State_prime * v_Old[carA,carB]  
        temp += pol_a_s * ( r_a + y * temp_a)
    return temp

In [ ]:
def updateV_Synchron(v_Old,policiy):
    v_new = np.zeros((21,21))
    for stateA in range(len(v_Old)):
        for stateB in range(len(v_Old[stateA])):
            state = (stateA,stateB)
            v_new[stateA,stateB] = updateV(state,v_Old,policiy)
    return v_new
        

In [ ]:
test = updateV_Synchron(v_old,policiy)

In [ ]:
test

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

MAX_CAPACITY = 20

def plot3d_over_states(f, zlabel="", ):
    A = np.arange(0, MAX_CAPACITY+1)
    B = np.arange(0, MAX_CAPACITY+1)
    # B, A !!!
    B, A = np.meshgrid(B, A)
    V = f.reshape(MAX_CAPACITY+1,-1)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    #ax = fig.gca(projection='3d')
    #surf = ax.plot_surface(A, B, V, rstride=1, cstride=1, cmap=cm.coolwarm,
    #                   linewidth=0, antialiased=False)
    #fig.colorbar(surf, shrink=0.5, aspect=5)
    ax.scatter(A, B, V, c='b', marker='.')
    ax.set_xlabel("cars at A")
    ax.set_ylabel("cars at B")
    ax.set_zlabel(zlabel)
 
    #ax.view_init(elev=10., azim=10)
    
    plt.show()

In [ ]:
plot3d_over_states(test) 

In [ ]:
v_old = np.zeros((21,21)) 
policiy = np.zeros((21,21,len(actions)))
policiy.fill(1/len(actions))

v_new  = updateV_Synchron(v_old,policiy)
plot3d_over_states(v_new ) 

v_ev10steps = list()
for i in range(3):
    for j in range(10):
        v_new = updateV_Synchron(v_new,policiy)
    v_ev10steps.append(v_new)

In [ ]:
plot3d_over_states(v_ev10steps[2])

In [ ]:
def oneStepLookAHead(s,v_new):
    # s ist (a,b)
    # γ = 1 = y
    # Fuer alle Aktionen expected Reward berechnen
    y = 1
    temp = 0 # sum
    actions = np.arange(-5,6,1)
    q_a = np.zeros(len(actions))
    for i in range(len(actions)):
        act = actions[i] 
        stateprobs, r_a = jack.get_transition_probabilities_and_expected_reward(s,act)
        # Summe ueber alle S'
        temp_a = 0
        for carA in range(len(stateprobs[0])):
            for carB in range(len(stateprobs[1])):
                prob_of_State_prime = stateprobs[0][carA] * stateprobs[1][carB]
                temp_a += prob_of_State_prime * v_new[carA,carB]  
        q_a[i] = r_a + y * temp_a
        
    best_actIndex = np.argmax(q_a)
    best_act = actions[best_actIndex]
    
    return best_act, best_actIndex

In [ ]:
z = np.asarray([0,1,3,2])

In [ ]:
np.argmax(z)

In [ ]:
# Improve policy by acting greedy
def beGreedy(v_new):
    policiy_new = np.zeros((21,21,len(actions)))
    carA, carB = 21, 21
    for carA_I in range(carA):
        for carB_I in range(carB):
            best_act, best_actIndex = oneStepLookAHead((carA_I,carB_I),v_new)
            policiy_new[carA_I,carB_I,best_actIndex] = 1
    return policiy_new
     

In [ ]:
v_new = v_ev10steps[2]

In [ ]:
policiy = beGreedy(v_new)
v_new = updateV_Synchron(v_new,policiy)

In [ ]:
plot3d_over_states(v_new)

In [ ]:
policiy[10,10]

In [ ]:
policiy = beGreedy(v_new)

In [ ]:
policiy[10,10]

In [ ]:
def plot_policy(policy):
    MAX_CAPACITY = 20
    A = np.arange(0, MAX_CAPACITY+1)
    B = np.arange(0, MAX_CAPACITY+1)
    A, B = np.meshgrid(A, B)
    Po = policy.reshape(MAX_CAPACITY+1,-1)
    levels = range(-5,6,1)
    plt.figure(figsize=(7,6))
    CS = plt.contourf(A, B, Po, levels)
    cbar = plt.colorbar(CS)
    cbar.ax.set_ylabel('actions')
    #plt.clabel(CS, inline=1, fontsize=10)
    plt.title('Policy')
    plt.xlabel("cars at B")
    plt.ylabel("cars at A")

In [ ]:
policiy.reshape(MAX_CAPACITY+1,-1)

In [ ]:
p = np.argmax(policiy,axis=2)

In [ ]:
plot_policy(p-5)

In [ ]:
policiy[5,20]

In [ ]:
p - 5